<a href="https://colab.research.google.com/github/ryuseunghan/IssueTracker/blob/main/content_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00


In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.2 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
def preprocessing(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])

    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()

    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()

    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()

    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()

    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]

    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()

    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()

    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])

    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()

    # 개행 문자 "\n" 제거
    text = text.replace('\n', '')

    return text

In [ ]:
import csv
from datetime import datetime

start_date = datetime.strptime('2023-07-15', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-07-17', '%Y-%m-%d').date()  # 기간 설정, 문서 수 169 약 30분
file_path = '/content/drive/MyDrive/이슈트래커/query_result.csv'
text = ''
final_text = ''

with open(file_path, 'r', encoding='utf-8') as f:
    data = csv.reader(f, delimiter=',')
    header = next(data)
    for row in data:
      date_string = row[3]  # Assuming the date is in the second column (index 1)
      row_date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S').date()
      if start_date <= row_date <= end_date:
        text = row[2]
        text = preprocessing(text)
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids[:510] + [tokenizer.eos_token_id]
        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
        summarized_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        final_text += summarized_text

In [ ]:
from konlpy.tag import Okt
import nltk

t = Okt()

const_doc =final_text.replace('\n', '')
tokens_const = t.nouns(final_text)
tokens_const = [token for token in tokens_const if len(token) > 1]
stopwords = ['포토'] # 포토 제거
clean_tokens_const = []

for token in tokens_const:
  if token not in stopwords:
    clean_tokens_const.append(token)
f.close()

In [ ]:
const_cnt = {}
max_words = 20

for word in clean_tokens_const:
    const_cnt[word] = const_cnt.get(word, 0) + 1
sorted_w = sorted(const_cnt.items(), key=lambda kv: kv[1])
result = sorted_w[-max_words:]
result.reverse()
print(result)

[('올스타', 284), ('경기', 160), ('올스타전', 155), ('신한은행', 150), ('부산', 143), ('사직야구장', 141), ('나눔', 136), ('드림', 115), ('리그', 71), ('기념', 50), ('촬영', 47), ('수원', 43), ('전북', 41), ('앞서', 39), ('북부', 30), ('홈런레이스', 28), ('프로야구', 28), ('컴투스', 27), ('홈런', 27), ('하나원', 26)]


In [ ]:
final_text

"15일 부산 사직야구장에서 열린 ‘2023 신한은행 SOL KBO 올스타전’ 드림 올스타와 나눔 올스타의 경기에서 박세웅이 선발투수 부문 베스트12로 뽑혔다.대한축구협회 심판위원회가 12일 ‘하나원큐 K리그1 2023’ 22라운드 인천 유나이티드와 홈경기 도중 나온 울산 현대 이규성(29)의 비신사적 행위를 놓고 내린 결론은 ‘공 없는 상황에서 노골적 안면 가격은 퇴장감이 아니다’라는 선례를 남겼다는 것이다.17일(이하 한국시각) 메이저리그 공식 홈페이지 MLB.com은 가장 긴 이름의 선수인 크리스티안 엔카나시온-스트랜드(24, 신시내티 레즈)가 메이저리그 데뷔를 앞두고 있다고 언급했다.7월 17일, 삼성서울병원장례식장 1호, 발인 19일 오전 5시 30분, 장지 성남영생원-동화경모공원[스포츠동아]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           샌디에이고 파드리스 김하성은 17일(한국시간) 시티즌스뱅크파크에서 열린 필라델피아 필리스와 원정경기에 1번타자 2루수로 선발출전해 6타수 2안타 1홈런 1타점 2득점의 활약을 펼쳤으며 시즌 타율은 0.262(294타수 77안타)로